In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
from datetime import timedelta
import numpy as np
from scipy import stats

In [2]:
# load large cap stocks price data and benchmark index data
p_df = pd.read_excel('price_data.xlsx')
df = p_df.copy()
df.reset_index(inplace=True)
df1 = df.T
df2 = df1.iloc[1:,:]
df2.reset_index(inplace=True)
cols = df2.iloc[0,:]
df2.columns = cols
df3 = df2.iloc[1:,:]
df3.rename(columns={'Unnamed: 0':'Date'},inplace=True)
df3.set_index(pd.to_datetime(df3.Date),inplace=True)

del df3['Date']
df3.reset_index(inplace=True)
pricedf = df3.copy()

C:\Users\HP\AppData\Local\Temp\ipykernel_39756\71662434.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.rename(columns={'Unnamed: 0':'Date'},inplace=True)


In [3]:
quarter_df =  pd.read_excel("quarterly_periods.xlsx")
value_reader = pd.ExcelFile('nse300_stocks.xlsx')

In [4]:
def get_volatility(tmpdf):
    volatility = ((tmpdf.pct_change().std()) * np.sqrt(252))*100
    return np.round(volatility,2)

In [5]:
def get_beta(tmpdf,ticker):
    tmpdf = tmpdf.reset_index()
    tmpdf.rename(columns={'date':'Date'},inplace=True)
    benchmark = pd.read_csv('BSE100.csv',index_col=False)
    
    benchmark['Date'] = pd.to_datetime(benchmark['Date'])
    benchmark = benchmark[['Date','Close']]
    
    merged_df = pd.merge(benchmark,tmpdf,on='Date')
    x = merged_df[ticker].pct_change()
    y = merged_df['Close'].pct_change()

    x.dropna(inplace=True)
    y.dropna(inplace=True)
    regress_coefs = stats.linregress(x,y)
    return round(regress_coefs.slope,2)


In [6]:
def get_cumulative_range(tmpdf):
    max_price = tmpdf.max()
    min_price = tmpdf.min()
    cum_range = max_price/min_price
    return np.round(cum_range,2)
    

In [7]:
def calc_zscore(bpdf):
    zscore_list=[]
    for factor in vol_factor_weights_dict.keys():
        bpdf[f'{factor}_mean'] = bpdf[factor].mean()
        bpdf[f'{factor}_std'] = bpdf[factor].std()
        bpdf[f'{factor}_zscore'] = ((bpdf[factor]-bpdf[f'{factor}_mean'])/bpdf[f'{factor}_std']) * vol_factor_weights_dict[factor]
        zscore_list.append(f'{factor}_zscore')
    bpdf['z_score_avg'] = bpdf[zscore_list].sum(axis=1) /len(zscore_list)
    bpdf['vol_score'] = np.where((bpdf.z_score_avg>0), (1+bpdf.z_score_avg), (1/(1-bpdf.z_score_avg)))
    return bpdf.copy()

In [8]:
vol_factor_weights_dict = {'volatility':0.38, 'capm_beta':0.27, 'cumulative_range':0.35}
writer = pd.ExcelWriter('volatility_factor_rank_nse300.xlsx')
#isin = pd.read_excel('largecap_isin.xlsx')
for rowidx, rowval in quarter_df.iterrows():
    date_range= rowval.Date
    quarter = rowval.quarters
    if(quarter=='Q12023'):
        break
    
    print(date_range, quarter)
    start_date = datetime.strptime(date_range.split(" - ")[0],"%d/%m/%Y")
    end_date = datetime.strptime(date_range.split(" - ")[1],"%d/%m/%Y")
    print(start_date, end_date)
    year_end_date = end_date # - timedelta(days=365)
    year_start_date = year_end_date -timedelta(days=365)
    print(year_start_date,year_end_date)
    quarter1 = f"{end_date.strftime('%Y%m')}"
    valuedf = pd.read_excel(value_reader, sheet_name=(quarter1))

    
    stock_list = valuedf.iloc[:,0].values
    #stock_list = pd.Series(stock_list)
    stock_lst = []
    for stock in stock_list:
        if(stock in pricedf.columns):
            stock_lst.append(stock)

    
    price_df = pricedf[pricedf['Date'].between(year_start_date, year_end_date)].reset_index(drop=True).sort_values('Date').set_index('Date')[stock_lst]

    price_df = price_df.T.drop_duplicates(keep='first').T
    
    row_lst = []
    for ticker in price_df.columns:
        tmpdf = price_df[ticker].dropna()[1:]
        if len(tmpdf>60):
            
            row_lst.append([ticker,get_volatility(tmpdf),get_beta(tmpdf,ticker),get_cumulative_range(tmpdf)])
    
    
    returns_df = pd.DataFrame(row_lst,columns=['Ticker','volatility','capm_beta','cumulative_range'])
    #returns_df.columns = returns_df.iloc[0]
    #returns_df.drop(0,inplace=True)
    rank_df = calc_zscore(returns_df)
    
    rank_df.sort_values('vol_score', inplace=True)
    rank_df.reset_index(drop=True,inplace=True)
    rank_df['rank'] = np.arange(1,len(returns_df)+1)
    
    rank_df.to_excel(writer, sheet_name=quarter1, index=False)
    
writer.save()

1/4/2022 - 30/6/2022 Q12022
2022-04-01 00:00:00 2022-06-30 00:00:00
2021-06-30 00:00:00 2022-06-30 00:00:00
1/7/2022 - 30/9/2022 Q22022
2022-07-01 00:00:00 2022-09-30 00:00:00
2021-09-30 00:00:00 2022-09-30 00:00:00
3/10/2022 - 30/12/2022 Q32022
2022-10-03 00:00:00 2022-12-30 00:00:00
2021-12-30 00:00:00 2022-12-30 00:00:00
2/1/2023 - 31/3/2023 Q42022
2023-01-02 00:00:00 2023-03-31 00:00:00
2022-03-31 00:00:00 2023-03-31 00:00:00


C:\Users\HP\AppData\Local\Temp\ipykernel_39756\854839179.py:58: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
